In [2]:
!hostnamectl

 Static hostname: DGX
       Icon name: computer-server
         Chassis: server 🖳
      Machine ID: dedf5691225742ff95b33f842b0781cc
         Boot ID: cb04df0ff1774eeda112c63867eeefdd
Operating System: ]8;;https://www.ubuntu.com/Ubuntu 24.04.2 LTS]8;;              
          Kernel: Linux 6.8.0-53-generic
    Architecture: x86-64
 Hardware Vendor: NVIDIA
  Hardware Model: DGXA100 920-23687-2530-002
Firmware Version: 1.29
   Firmware Date: Tue 2024-08-06
    Firmware Age: 7month 3w 3d


In [3]:
%cd /home/ir739wb/ilyarekun/bc_project/resnet18-centralyzed-test/scripts

/home/ir739wb/ilyarekun/bc_project/resnet18-centralyzed-test/scripts


In [4]:
%ls

__pycache__/  data1.py    model.py               results.txt
data.ipynb    main.ipynb  model_centralyzed.pth


In [5]:
import os
import sys
sys.path.append('../scripts')

import time
import shutil
import kagglehub
import torch
import torchvision
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
import random
import skimage
from PIL import Image, ImageFile
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models import ResNet18_Weights
from torchvision.datasets import ImageFolder
from data1 import data_preprocessing_tumor


In [6]:
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
random.seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic=True
torch.backends.cudnn.benchmark=False


In [7]:
import os

def count_files(folder_path):
    return len(os.listdir(folder_path))

test_path = "/home/ir739wb/ilyarekun/bc_project/data/9/test"
val_path = "/home/ir739wb/ilyarekun/bc_project/data/9/val"
train_path = "/home/ir739wb/ilyarekun/bc_project/data/9/train"

print("Test files:", count_files(test_path))
print("Validation files:", count_files(val_path))
print("Train files:", count_files(train_path))


Test files: 8482
Validation files: 8473
Train files: 67863


In [8]:
EPOCHS = 100
BATCH_SIZE = 64
TOTAL_SIZE = 4916
TOTAL_TEST_SIZE = 703
TOTAL_VAL_SIZE = 1404
STEPS_PER_EPOCH = TOTAL_SIZE // BATCH_SIZE
STEPS_PER_TEST_EPOCH = TOTAL_TEST_SIZE // BATCH_SIZE
STEPS_PER_VAL_EPOCH = TOTAL_VAL_SIZE // BATCH_SIZE
IMAGE_H, IMAGE_W = 224, 224

In [9]:
class CustomModel(nn.Module):
    def __init__(self, base_model, num_features, num_classes):
        super(CustomModel, self).__init__()
        self.base_model = base_model
        self.fc_layers = nn.Sequential(
            nn.Linear(num_features, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )
    def forward(self, x):
        x = self.base_model(x)
        x = self.fc_layers(x)
        return x



In [10]:
train_loader, val_loader, test_loader = data_preprocessing_tumor()

BATCH_SIZE = 64
IMAGE_H, IMAGE_W = 224, 224
NUM_WORKERS = 4

In [11]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

In [12]:
model_ft = torchvision.models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
#model_ft.avgpool = Identity()
num_ftrs = model_ft.fc.in_features  # 512
""" model_ft.fc = nn.Sequential(
    nn.Linear(num_ftrs, 256),
    nn.BatchNorm1d(256),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(256, 4)
) """



model_ft.fc = nn.Sequential(
    nn.Linear(num_ftrs, 512),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.Dropout(p=0.7),
    nn.Linear(512, 256),
    nn.BatchNorm1d(256),
    nn.ReLU(),
    nn.Dropout(p=0.7),
    nn.Linear(256, 128),
    nn.BatchNorm1d(128),
    nn.ReLU(),
    nn.Dropout(p=0.7),
    nn.Linear(128, 64),
    nn.BatchNorm1d(64),
    nn.ReLU(),
    nn.Dropout(p=0.3),
    nn.Linear(64, 4)
)



In [13]:
print(model_ft)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [14]:


#for param in model_ft.parameters():
#    param.requires_grad = False

#for param in model_ft.layer3.parameters():
#    param.requires_grad = True
for param in model_ft.layer4.parameters():  
    param.requires_grad = True
    


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft.to(device)

class_weights = torch.tensor([1.0,1.0 ]).to(device)
criterion = nn.CrossEntropyLoss()


optimizer = torch.optim.Adam([
    {"params": model_ft.fc.parameters(), "lr": 0.001},
    #{"params": model_ft.layer3.parameters(), "lr": 0.0005},
    {"params": model_ft.layer4.parameters(), "lr": 0.0005},
], weight_decay=1e-3)
 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=8, gamma=0.7)

if torch.cuda.device_count() > 1:
    print("Using DataParallel for multi-GPU training")
    model_ft = nn.DataParallel(model_ft)

In [15]:
from sklearn.metrics import precision_score, recall_score, f1_score

def get_test():
    test_loss = []
    all_preds = []
    all_targets = []

    for batch_idx, (data, target) in enumerate(test_loader):
        if batch_idx == STEPS_PER_TEST_EPOCH:
            break

        with torch.no_grad():
            data = data.to(device)
            target = target.long().to(device)
            output = model_ft(data)
            criterion = nn.CrossEntropyLoss()
            loss = criterion(output, target)

            # Save all predictions and true labels for metric calculation
            preds = output.argmax(dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(target.cpu().numpy())

        test_loss.append(float(loss.detach()))

    # Handle the case when no samples are processed
    if len(all_targets) > 0:
        correct = sum([1 for p, t in zip(all_preds, all_targets) if p == t])
        incorrect = len(all_targets) - correct
        accuracy = correct / len(all_targets)
        precision = precision_score(all_targets, all_preds, average='macro', zero_division=0)
        recall = recall_score(all_targets, all_preds, average='macro', zero_division=0)
        f1 = f1_score(all_targets, all_preds, average='macro', zero_division=0)
    else:
        correct = 0
        incorrect = 0
        accuracy = 0
        precision = 0
        recall = 0
        f1 = 0

    # Handle the case when no losses are recorded
    if len(test_loss) > 0:
        avg_test_loss = sum(test_loss) / len(test_loss)
    else:
        avg_test_loss = 0  # Default value when no test steps occur

    print(f"CORRECT: {correct}, INCORRECT: {incorrect}, TEST ACCURACY: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

    return accuracy, avg_test_loss, precision, recall, f1

In [16]:
from sklearn.metrics import precision_score, recall_score, f1_score

def get_validation():
    val_loss = []
    all_preds = []
    all_targets = []

    for batch_idx, (data, target) in enumerate(val_loader):
        if batch_idx == STEPS_PER_VAL_EPOCH:
            break

        with torch.no_grad():
            data = data.to(device)
            target = target.long().to(device)
            output = model_ft(data)
            criterion = nn.CrossEntropyLoss()
            loss = criterion(output, target)

            # Save all predictions and true labels for metric calculation
            preds = output.argmax(dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(target.cpu().numpy())

        val_loss.append(float(loss.detach()))

    # Handle the case when no samples are processed
    if len(all_targets) > 0:
        correct = sum([1 for p, t in zip(all_preds, all_targets) if p == t])
        incorrect = len(all_targets) - correct
        accuracy = correct / len(all_targets)
        precision = precision_score(all_targets, all_preds, average='macro', zero_division=0)
        recall = recall_score(all_targets, all_preds, average='macro', zero_division=0)
        f1 = f1_score(all_targets, all_preds, average='macro', zero_division=0)
    else:
        correct = 0
        incorrect = 0
        accuracy = 0
        precision = 0
        recall = 0
        f1 = 0

    # Handle the case when no losses are recorded
    if len(val_loss) > 0:
        avg_val_loss = sum(val_loss) / len(val_loss)
    else:
        avg_val_loss = 0  # Default value when no validation steps occur

    print(f"CORRECT: {correct}, INCORRECT: {incorrect}, VAL ACCURACY: {accuracy:.4f}")
    return accuracy, avg_val_loss, precision, recall, f1

In [17]:
%ls 

__pycache__/  data1.py    model.py               results.txt
data.ipynb    main.ipynb  model_centralyzed.pth


In [18]:
# Использование функции для вывода результатов
acc_, loss_, precision_, recall_, f1_ = get_test()
print("ACCURACY AND LOSS BEFORE TUNING")
print(f"ACCURACY: {acc_:.4f}, LOSS: {loss_:.4f}")
print(f"Precision: {precision_:.4f}, Recall: {recall_:.4f}, F1: {f1_:.4f}")


CORRECT: 155, INCORRECT: 485, TEST ACCURACY: 0.2422
Precision: 0.2244, Recall: 0.2395, F1: 0.1770
ACCURACY AND LOSS BEFORE TUNING
ACCURACY: 0.2422, LOSS: 1.5040
Precision: 0.2244, Recall: 0.2395, F1: 0.1770


In [19]:
print(model_ft)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [21]:
import subprocess
subprocess.run(["nvidia-smi", "-L"])

GPU 0: NVIDIA A100-SXM4-40GB (UUID: GPU-ff3ba929-da03-8441-6ab6-0ad20ecbac5f)


CompletedProcess(args=['nvidia-smi', '-L'], returncode=0)

In [20]:
avg_val_loss_history = []
avg_val_accuracy_history = []
avg_train_loss_history = []
avg_train_accuracy_history = []

loss_history = []
accuracy_history = []

new_best = 0

for epoch in range(EPOCHS):
    start = time.time()
    print(f"-----------------------EPOCH {epoch} -----------------------------------")
    
    for batch_idx, (data, target) in enumerate(train_loader):
        if batch_idx == STEPS_PER_EPOCH:
            break
        
        optimizer.zero_grad()
        data = data.to(device)
        target = target.to(device)
        output = model_ft(data)
        loss = criterion(output, target.reshape((BATCH_SIZE,)).long())
        loss_history.append(loss.detach())
        loss.backward()
        optimizer.step()
        
        # Calculate the number of correct predictions
        correct = 0
        for p in range(BATCH_SIZE):
            # Convert logits to a list of numbers
            a = [float(j.detach()) for j in output[p]]
            pred = a.index(max(a))
            if pred == int(target[p]):
                correct += 1
        
        batch_accuracy = correct / BATCH_SIZE
        accuracy_history.append(batch_accuracy)
        
        print(
            f"\r EPOCH {epoch} MINIBATCH: {batch_idx}/{STEPS_PER_EPOCH} LOSS: {loss_history[-1]:.4f}",
            end=""
        )
    
    end = time.time()
    
    avg_train_loss = sum(loss_history[-STEPS_PER_EPOCH:]) / STEPS_PER_EPOCH
    avg_train_acc = sum(accuracy_history[-STEPS_PER_EPOCH:]) / STEPS_PER_EPOCH
    
    avg_train_loss_history.append(avg_train_loss)
    avg_train_accuracy_history.append(avg_train_acc)
    
    # Get validation metrics; it is assumed that the function returns 5 values
    val_acc, val_loss, precision, recall, f1 = get_validation()
    avg_val_accuracy_history.append(val_acc)
    avg_val_loss_history.append(val_loss)
    
    scheduler.step()  # Update learning rate
    
    if val_acc > new_best:
        new_best = val_acc
        torch.save(model_ft.state_dict(), "model_centralyzed.pth")
    
    # Form the result string for output and writing to file
    str_res = (f"Epoch {epoch+1}/{EPOCHS}, Train Loss: {avg_train_loss:.4f}, "
               f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc*100:.2f}%, "
               f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    print("\n" + str_res)
    with open('results.txt', 'a') as f:
        f.write(str_res + "\n")
    
    print(f"Epoch time: {end - start:.2f} sec")

-----------------------EPOCH 0 -----------------------------------
 EPOCH 0 MINIBATCH: 75/76 LOSS: 0.6739CORRECT: 1034, INCORRECT: 310, VAL ACCURACY: 0.7693

Epoch 1/100, Train Loss: 1.0265, Val Loss: 0.6996, Val Acc: 76.93%, Precision: 0.7578, Recall: 0.7558, F1: 0.7405
Epoch time: 17.43 sec
-----------------------EPOCH 1 -----------------------------------
 EPOCH 1 MINIBATCH: 75/76 LOSS: 0.3243CORRECT: 1184, INCORRECT: 160, VAL ACCURACY: 0.8810

Epoch 2/100, Train Loss: 0.5256, Val Loss: 0.4151, Val Acc: 88.10%, Precision: 0.8748, Recall: 0.8749, F1: 0.8746
Epoch time: 16.12 sec
-----------------------EPOCH 2 -----------------------------------
 EPOCH 2 MINIBATCH: 75/76 LOSS: 0.1983CORRECT: 1239, INCORRECT: 105, VAL ACCURACY: 0.9219

Epoch 3/100, Train Loss: 0.2797, Val Loss: 0.2887, Val Acc: 92.19%, Precision: 0.9190, Recall: 0.9191, F1: 0.9186
Epoch time: 16.08 sec
-----------------------EPOCH 3 -----------------------------------
 EPOCH 3 MINIBATCH: 56/76 LOSS: 0.1412

KeyboardInterrupt: 

In [ ]:
train_dataset = train_loader.dataset  # Получаем датасет из лоадера
class_to_idx = train_dataset.label_map  # Используем label_map из ChestXRayDataset
print(class_to_idx)  # {'positive': 1, 'negative': 0}


In [ ]:
print("ACCURACY : ",new_best)

In [ ]:
plt.plot(avg_train_loss_history,label = "Train")
plt.plot(avg_val_loss_history , label = "Test")
plt.title('LOSS PER EPOCH')
plt.xlabel("EPOCHS")
plt.ylabel("LOSS")

In [ ]:
plt.plot(loss_history)
plt.title('LOSS PER BATCH')
plt.xlabel("BATCH")
plt.ylabel("LOSS")
plt.show()

In [ ]:
plt.plot(avg_train_accuracy_history , label = "Train")
plt.plot(avg_test_accuracy_history , label = "Test")
plt.title('ACCURACY PER EPOCH')
plt.xlabel("EPOCHS")
plt.ylabel("ACCURACY")